<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/01_Avg%2C_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
import warnings
import re
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [81]:
df = pd.read_csv('car_prices.csv')
df.shape

(54273, 13)

In [82]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [83]:
for col in df.columns:
    print(f'{col}: {df[col].nunique()}')

id: 54273
brand: 53
model: 1827
model_year: 34
milage: 3212
fuel_type: 7
engine: 1061
transmission: 46
ext_col: 260
int_col: 124
accident: 2
clean_title: 1
price: 1481


In [84]:
def parse_engine_info(engine):
    if pd.isna(engine):
        return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan])

    hp = re.search(r'(\d+\.?\d*)HP', engine)
    liter = re.search(r'(\d+\.?\d*)L', engine)
    motor = re.search(r'(Straight|V)\s*\d*', engine)
    cylinder = re.search(r'(\d+)\s*Cylinder', engine)

    return pd.Series([
        hp.group(1) if hp else np.nan,
        liter.group(1) if liter else np.nan,
        motor.group(0) if motor else np.nan,
        cylinder.group(1) if cylinder else np.nan,
    ])

df[['hp', 'litr', 'motor', 'Cylinder']] = df['engine'].apply(parse_engine_info)

In [85]:
df = df.drop(columns=['clean_title', 'id'])

In [86]:
df.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,hp,litr,motor,Cylinder
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,11000,375.0,3.5,V6,6


In [87]:
df.isna().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0
accident,0


In [125]:
def fill_nan_mean(df, col, columns):
    print(f"Col: {col}, Columns: {columns}, Nan: {df[col].isna().sum()}", end='')
    value = df[df[col].notna()].groupby(by=columns)[col].mean().reset_index()
    df = pd.merge(df, value, on=columns, how='left', suffixes=('', '_mean'))
    df[col] = df[col].fillna(df[col + '_mean'])
    df.drop(columns=[col + '_mean'], inplace=True)
    print(f", After Clean: {df[col].isna().sum()}")
    return df


def fill_nan_mode(df, col, columns):
    print(f"Col: {col}, Columns: {columns}, Nan: {df[col].isna().sum()}", end='')
    mode_values = df.groupby(columns)[col].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA).reset_index()
    df = pd.merge(df, mode_values, on=columns, how='left', suffixes=('', '_mode'))
    df[col] = df[col].fillna(df[col + '_mode'])
    df.drop(columns=[col + '_mode'], inplace=True)

    print(f", After Clean: {df[col].isna().sum()}")
    return df

def to_numeric(x):
    try:
        return float(x)
    except ValueError as err:
        print(err)
        return np.nan

In [95]:
df['hp'] = df['hp'].agg(to_numeric)

df = fill_nan_mean(df.copy(), 'hp', ['motor', 'Cylinder', 'litr'])
df = fill_nan_mean(df.copy(), 'hp', ['motor', 'litr'])
df = fill_nan_mean(df.copy(), 'hp', ['brand', 'model'])
df = fill_nan_mean(df.copy(), 'hp', ['litr', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'hp', ['model'])
df = fill_nan_mean(df.copy(), 'hp', ['motor'])
df = fill_nan_mean(df.copy(), 'hp', ['price', 'brand'])
df = fill_nan_mean(df.copy(), 'hp', ['price', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'hp', ['engine'])



print(df['hp'].isna().sum())

Col: hp, Columns: ['motor', 'Cylinder', 'litr'], Nan: 4057, After Nan: 4055
Col: hp, Columns: ['motor', 'litr'], Nan: 4055, After Nan: 2809
Col: hp, Columns: ['brand', 'model'], Nan: 2809, After Nan: 369
Col: hp, Columns: ['litr', 'fuel_type'], Nan: 369, After Nan: 119
Col: hp, Columns: ['model'], Nan: 119, After Nan: 111
Col: hp, Columns: ['motor'], Nan: 111, After Nan: 107
Col: hp, Columns: ['price', 'brand'], Nan: 107, After Nan: 23
Col: hp, Columns: ['price', 'fuel_type'], Nan: 23, After Nan: 14
Col: hp, Columns: ['engine'], Nan: 14, After Nan: 0
0


In [96]:
df['litr'] = df['litr'].agg(to_numeric)


df = fill_nan_mean(df.copy(), 'litr', ['motor', 'Cylinder', 'hp'])
# df = fill_nan_mean(df.copy(), 'litr', ['Cylinder', 'hp'])
# df = fill_nan_mean(df.copy(), 'litr', ['motor', 'hp'])
df = fill_nan_mean(df.copy(), 'litr', ['hp', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'litr', ['model'])
df = fill_nan_mean(df.copy(), 'litr', ['engine'])
df = fill_nan_mean(df.copy(), 'litr', ['price', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'litr', ['price', 'transmission'])


print(df['litr'].isna().sum())

Col: litr, Columns: ['motor', 'Cylinder', 'hp'], Nan: 606, After Nan: 606
Col: litr, Columns: ['hp', 'fuel_type'], Nan: 606, After Nan: 396
Col: litr, Columns: ['model'], Nan: 396, After Nan: 124
Col: litr, Columns: ['engine'], Nan: 124, After Nan: 46
Col: litr, Columns: ['price', 'fuel_type'], Nan: 46, After Nan: 3
Col: litr, Columns: ['price', 'transmission'], Nan: 3, After Nan: 0
0


In [98]:
df['Cylinder'] = df['Cylinder'].agg(to_numeric)

df = fill_nan_mean(df.copy(), 'Cylinder', ['litr', 'hp'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['litr', 'model'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['hp', 'model'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['model'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['hp', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['price', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['price', 'transmission'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['litr', 'fuel_type'])
df = fill_nan_mean(df.copy(), 'Cylinder', ['engine'])


print(df['Cylinder'].isna().sum())

Col: Cylinder, Columns: ['litr', 'hp'], Nan: 4175, After Nan: 3772
Col: Cylinder, Columns: ['litr', 'model'], Nan: 3772, After Nan: 1383
Col: Cylinder, Columns: ['hp', 'model'], Nan: 1383, After Nan: 1033
Col: Cylinder, Columns: ['model'], Nan: 1033, After Nan: 632
Col: Cylinder, Columns: ['hp', 'fuel_type'], Nan: 632, After Nan: 214
Col: Cylinder, Columns: ['price', 'fuel_type'], Nan: 214, After Nan: 32
Col: Cylinder, Columns: ['price', 'transmission'], Nan: 32, After Nan: 8
Col: Cylinder, Columns: ['litr', 'fuel_type'], Nan: 8, After Nan: 2
Col: Cylinder, Columns: ['engine'], Nan: 2, After Nan: 0
0


In [109]:
df.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'price', 'hp', 'litr',
       'motor', 'Cylinder'],
      dtype='object')

In [113]:
df.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,hp,litr,motor,Cylinder
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,11000,375.0,3.5,V6,6.0


In [129]:
new_df = fill_nan_mode(df.copy(), 'motor', ['litr', 'hp', 'Cylinder', 'model'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['litr', 'hp', 'Cylinder'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['litr', 'hp'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['model'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['engine'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['fuel_type', 'hp'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['litr', 'Cylinder'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['Cylinder', 'price'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['Cylinder', 'brand'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['litr', 'brand'])
new_df = fill_nan_mode(new_df.copy(), 'motor', ['hp', 'brand'])


print(new_df['motor'].isna().sum())

Col: motor, Columns: ['litr', 'hp', 'Cylinder', 'model'], Nan: 29640, After Clean: 28986
Col: motor, Columns: ['litr', 'hp', 'Cylinder'], Nan: 28986, After Clean: 21847
Col: motor, Columns: ['litr', 'hp'], Nan: 21847, After Clean: 21438
Col: motor, Columns: ['model'], Nan: 21438, After Clean: 6649
Col: motor, Columns: ['engine'], Nan: 6649, After Clean: 1320
Col: motor, Columns: ['fuel_type', 'hp'], Nan: 1320, After Clean: 522
Col: motor, Columns: ['litr', 'Cylinder'], Nan: 522, After Clean: 76
Col: motor, Columns: ['Cylinder', 'price'], Nan: 76, After Clean: 66
Col: motor, Columns: ['Cylinder', 'brand'], Nan: 66, After Clean: 52
Col: motor, Columns: ['litr', 'brand'], Nan: 52, After Clean: 40
Col: motor, Columns: ['hp', 'brand'], Nan: 40, After Clean: 12
12


In [100]:
df.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,hp,litr,motor,Cylinder
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,11000,375.0,3.5,V6,6.0


In [101]:
df.isna().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0
accident,0


In [102]:
df['Cylinder'].unique()

array([ 6.        ,  8.        ,  4.        ,  4.5       ,  4.22641509,
        6.1465798 ,  5.        , 12.        ,  6.14185661, 10.        ,
        4.59475916,  5.83333333,  7.94326241,  4.8       ,  5.82371795,
        6.4962406 ,  6.00569486,  8.89473684,  7.33333333,  5.9178515 ,
        7.42857143,  6.08955224,  6.5       ,  7.08333333,  3.        ,
        4.09366613,  9.        ,  5.96792381,  5.6147541 ,  4.04245283,
        4.66666667,  7.76470588,  4.33333333,  4.03333333,  7.5       ,
        7.88235294,  5.84033613,  7.02364277,  5.93548387,  5.81818182,
        6.03571429,  7.2       ,  4.2       ,  4.44444444,  4.21226415,
        4.52898551,  6.13157895,  6.23504274,  7.27272727,  5.29491525,
        5.97740964,  6.1       ,  6.28571429,  7.        ,  6.02094241,
        7.27151878,  6.54545455,  5.75257732,  5.77249637,  6.01328904,
        4.39295877,  6.01724138,  8.125     ,  6.00113897,  5.85185185,
        6.79166667,  6.2       ,  4.19977802,  7.77777778,  5.97

In [103]:
# mean_prices = df[df['hp'].notna()].groupby(by=['Route', 'Date_of_Journey', 'Additional_Info'])['hp'].mean().reset_index()
# df = pd.merge(df, mean_prices, on=['Route', 'Date_of_Journey', 'Additional_Info'], how='left', suffixes=('', '_mean'))
# df['hp'] = df['hp'].fillna(df['Price_mean'])
# df.drop(columns=['Price_mean'], inplace=True)

In [104]:
df[col] = pd.to_numeric(df[col], errors='coerce')

In [105]:
df[col].isna().sum()

0

In [106]:
# mean_prices = df[df['Price'].notna()].groupby(by=['Route', 'Date_of_Journey', 'Additional_Info'])['Price'].mean().reset_index()
# df = pd.merge(df, mean_prices, on=['Route', 'Date_of_Journey', 'Additional_Info'], how='left', suffixes=('', '_mean'))
# df['Price'] = df['Price'].fillna(df['Price_mean'])
# df.drop(columns=['Price_mean'], inplace=True)

In [107]:
# df['hp'] = df['engine'].agg(lambda x: x.split('HP')[0] if "HP" in x else np.nan)
# df['hp'] = pd.to_numeric(df['hp'])

In [108]:
# df['litr'] = df['engine'].agg(lambda x: x.split('L').split(' ')[0] if "L" in x else np.nan)
# df['litr'] = pd.to_numeric(df['litr'])
# df.head()